# Practice for SQL Subqueries

Connect to the Database and import the appropriate libaries.

In [1]:
# Run this code without change

import sqlite3
import pandas as pd
conn = sqlite3.Connection('data (3).sqlite')

1. Write an Equivalent Query using a Subquery

The following query works using a JOIN. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [2]:
q = """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;"""
pd.read_sql(q, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [3]:
q = """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
WHERE customerNumber IN (SELECT customerNumber
                          FROM orders
                          WHERE orderDate = '2003-01-31')
;"""
pd.read_sql(q, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


2. Select the Total Number of Orders for Each Product Name
- Sort the results by the total number of items sold for that product.

productName
total_orders → how many orders included this product
total_items_sold → total quantity of this product sold

In [17]:
# CHATGPT

q = """
SELECT
    productName,
    total_orders,
    total_items_sold
FROM (
    SELECT
        p.productName,
        COUNT(DISTINCT od.orderNumber) AS total_orders,
        SUM(od.quantityOrdered) AS total_items_sold
    FROM orderdetails AS od
    JOIN products AS p ON od.productCode = p.productCode
    GROUP BY p.productName
) AS sub
ORDER BY total_items_sold DESC;
"""

pd.read_sql(q, conn)

,productName,total_orders,total_items_sold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803


3. Select the Product Name and the Total Number of People Who Have Ordered Each Product
- Sort the results in descending order.

In [25]:
q = """
SELECT
    productName,
    COUNT(DISTINCT customerNumber) AS total_customers
    FROM products
    LEFT JOIN orderdetails
        USING(productCode)
    LEFT JOIN orders
        USING(orderNumber)
    LEFT JOIN customers
        USING(customerNUmber)
    GROUP BY productName
;"""

pd.read_sql(q, conn)

,productName,total_customers
0,18th Century Vintage Horse Carriage,26
1,18th century schooner,24
2,1900s Vintage Bi-Plane,26
3,1900s Vintage Tri-Plane,22
4,1903 Ford Model A,21
...,...,...
105,The Mayflower,22
106,The Queen Mary,24
107,The Schooner Bluenose,23
108,The Titanic,22


In [27]:
# Brings NaN in items sold because there are products in the list that haven't been sold

q = """
SELECT
    productName,
    COUNT(DISTINCT customerNumber) AS total_customers,
    SUM(quantityOrdered) AS total_items_sold
    FROM products
    LEFT JOIN orderdetails
        USING(productCode)
    LEFT JOIN orders
        USING(orderNumber)
    LEFT JOIN customers
        USING(customerNUmber)
    GROUP BY productName
    ORDER BY total_customers DESC
;"""

pd.read_sql(q, conn)

,productName,total_customers,total_items_sold
0,1992 Ferrari 360 Spider red,40,1808.0
1,Boeing X-32A JSF,27,960.0
2,1972 Alfa Romeo GTA,27,1030.0
3,1952 Alpine Renault 1300,27,961.0
4,1934 Ford V8 Coupe,27,985.0
...,...,...,...
105,2002 Chevy Corvette,18,894.0
106,1969 Chevrolet Camaro Z28,18,870.0
107,1952 Citroen-15CV,18,873.0
108,1949 Jaguar XK 120,18,949.0


4. Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees who sold products that have been ordered by fewer than 20 people.
- Hint:  To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [40]:
q = """
SELECT
    DISTINCT e.employeeNumber,
    e.firstName,
    e.lastName,
    o.city,
    officeCode,
    COUNT(DISTINCT customerNumber) AS num_customers,
    COUNT(DISTINCT productCode) AS num_product
    FROM employees AS e
    LEFT JOIN customers
        ON employeeNumber = salesRepEmployeeNumber
    LEFT JOIN orders
        USING(customerNumber)
    LEFT JOIN orderdetails
        USING(orderNumber)
    LEFT JOIN products
        USING(productCode)
    LEFT JOIN offices o
        USING(officeCode)
    GROUP BY employeeNumber
    HAVING num_product < 20
    ORDER BY num_customers DESC
;"""

pd.read_sql(q, conn)

,employeeNumber,firstName,lastName,city,officeCode,num_customers,num_product


In [39]:
q = """
SELECT
    DISTINCT employeeNumber,
    officeCode,
    o.city,
    firstName,
    lastName
FROM employees AS e
JOIN offices AS o
    USING(officeCode)
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
WHERE productCode IN (
    SELECT productCode
    FROM products
    JOIN orderdetails
        USING(productCode)
    JOIN orders
        USING(orderNumber)
    GROUP BY productCode
    HAVING COUNT(DISTINCT customerNumber) < 20
)
;
"""
pd.read_sql(q, conn)

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1501,7,London,Larry,Bott
2,1337,4,Paris,Loui,Bondur
3,1166,1,San Francisco,Leslie,Thompson
4,1286,3,NYC,Foon Yue,Tseng
5,1612,6,Sydney,Peter,Marsh
6,1611,6,Sydney,Andy,Fixter
7,1401,4,Paris,Pamela,Castillo
8,1621,5,Tokyo,Mami,Nishi
9,1323,3,NYC,George,Vanauf


5. Select the Employee Number, First Name, Last Name, and Number of Customers for Employees whose customers have an average credit limit over 15K.

In [43]:
# My solution
# In the solution on ine 

q = """
SELECT
    employeeNumber,    
    firstName,
    lastName,
    COUNT(DISTINCT customerNumber) AS total_customers,
    AVG(creditLimit) AS avg_credit
FROM employees e
    JOIN customers c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    GROUP BY employeeNumber
    HAVING avg_credit > 15000
;"""

pd.read_sql(q, conn)

,employeeNumber,firstName,lastName,total_customers,avg_credit
0,1165,Leslie,Jennings,6,100433.333333
1,1166,Leslie,Thompson,6,65266.666667
2,1188,Julie,Firrelli,6,73916.666667
3,1216,Steve,Patterson,6,81533.333333
4,1286,Foon Yue,Tseng,7,66614.285714
5,1323,George,Vanauf,8,80887.500000
6,1337,Loui,Bondur,6,86233.333333
7,1370,Gerard,Hernandez,7,91785.714286
8,1401,Pamela,Castillo,10,81340.000000
9,1501,Larry,Bott,8,91187.500000
